In [43]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from langchain_core.documents import Document
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tiktoken
from tqdm import tqdm
import os

In [36]:
doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')
processed_content = doc['Processed_Content']

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\admin-npatel237\AppData\Local\Temp\ipykernel_1652\1769151165.py:1: SyntaxWarning: invalid escape sequence '\T'
  doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')


In [44]:
embedding_model = HuggingFaceEmbeddings(model_name="nlpaueb/legal-bert-base-uncased")

No sentence-transformers model found with name nlpaueb/legal-bert-base-uncased. Creating a new one with mean pooling.


In [38]:
def chunk_text(text, max_tokens=1024, overlap=128):
    tokenizer = tiktoken.get_encoding("cl100k_base")  # Same tokenizer as text-embedding-005
    tokens = tokenizer.encode(text)

    chunks = []
    start = 0
    while start < len(tokens):
        chunk = tokens[start:start + max_tokens]
        chunks.append(tokenizer.decode(chunk))
        start += max_tokens - overlap  # Sliding window
    return chunks

doc["Processed_Content"] = doc["Processed_Content"].apply(lambda x: chunk_text(x) if len(x) > 32000 else [x])
df_exploded = doc.explode("Processed_Content").reset_index(drop=True)



In [40]:
def get_embeddings(texts, batch_size=32):
    """
    Compute embeddings in batches using VertexAIEmbeddings in LangChain.
    Args:
        texts (list of str): List of text data to embed.
        batch_size (int): Number of texts to process per batch.

    Returns:
        list: List of embedding vectors.
    """
    embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):
        batch = texts[i:i + batch_size]  # Get batch of texts
        batch_embeddings = embedding_model.embed_documents(batch)  # Generate embeddings
        embeddings.extend(batch_embeddings)  # Store results

    return embeddings  # Returning list of lists (each embedding is a list of floats)

# Store embeddings as a list of lists in DataFrame
df_exploded["Embedding"] = get_embeddings(df_exploded["Processed_Content"].tolist())

Generating Embeddings: 100%|██████████| 52/52 [03:40<00:00,  4.25s/it]


In [41]:
df_exploded.to_parquet("embeddings_gemini_text-005.parquet", engine="pyarrow")

TESTING:

In [50]:
embeds = pd.read_parquet("embeddings_gemini_text-005.parquet", engine="pyarrow")

In [51]:
sample = "After reviewing the individual's financial disclosure report submitted in accordance with chapter 131 of title 5, the official in charge of the employee's appointment certifies in writing that the need for the individual's services outweighs the possibility of a conflict of interest brought about by the financial interest involved."

In [52]:
def chunk_text(text, max_tokens=1024, overlap=128):
    tokenizer = tiktoken.get_encoding("cl100k_base")  # Same tokenizer as text-embedding-005
    tokens = tokenizer.encode(text)

    chunks = []
    start = 0
    while start < len(tokens):
        chunk = tokens[start:start + max_tokens]
        chunks.append(tokenizer.decode(chunk))
        start += max_tokens - overlap  # Sliding window
    return chunks

sample = chunk_text(sample)

In [53]:
def get_embeddings(texts, batch_size=32):
    """
    Compute embeddings in batches using VertexAIEmbeddings in LangChain.
    Args:
        texts (list of str): List of text data to embed.
        batch_size (int): Number of texts to process per batch.

    Returns:
        list: List of embedding vectors.
    """
    embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):
        batch = texts[i:i + batch_size]  # Get batch of texts
        batch_embeddings = embedding_model.embed_documents(batch)  # Generate embeddings
        embeddings.extend(batch_embeddings)  # Store results

    return embeddings  # Returning list of lists (each embedding is a list of floats)

# Store embeddings as a list of lists in DataFrame

query_embedding = get_embeddings(sample)

Generating Embeddings: 100%|██████████| 1/1 [00:00<00:00, 17.27it/s]


In [56]:
query_embedding = np.array(query_embedding).reshape(1, -1)

# Convert the embeddings from a Pandas Series to a NumPy array
embeddings = np.vstack(embeds['Embedding'].values)  # Stack into a 2D array

print(type(embeddings))  # Should print <class 'numpy.ndarray'>

# Compute cosine similarity
similarities = cosine_similarity(query_embedding, embeddings)

# Get the index of the most similar document
most_similar_index = np.argmax(similarities)

print("Most similar document index:", most_similar_index)

<class 'numpy.ndarray'>
Most similar document index: 246
